In [ ]:
# This is R code in juptyer lab, to install R kernel in jupter, open R session inside your oen anaconda environment
# type in the following:
# install.packages("devtools") (fail)
# conda install -c conda-forge r-devtools (Use this)
# conda install -c conda-forge gcc
# devtools::install_github("IRkernel/IRkernel")
# IRkernel::installspec()
# then restart jupterlab
# ref: https://richpauloo.github.io/2018-05-16-Installing-the-R-kernel-in-Jupyter-Lab/
library(rxnorm)
library(dplyr)
library(data.table)
library(reshape2)
library(stringi)

In [ ]:
RIGHT = function(x,n){
  substring(x,nchar(x)-n+1)
}

In [ ]:
# read data
query_per_site_rx <- function(site){
    home_directory = "/home/hchan2/AKI/AKI_Python/"
    rxnorm_data_ext <- readRDS(paste0(home_directory,"rxnorm2atcdict.rda"))
    rxnorm_data_new <- read.delim(paste0(home_directory,'data/',site,"/rxnormtmp_",site,".csv"), sep=",", stringsAsFactors=F, header = FALSE)

    # find new query needed
    rxnorm_data_q <- rxnorm_data_new %>% filter(!V1 %in% rxnorm_data_ext$V1)

    #query new data
    rxnorm_data_new_q <- rxnorm_data_q %>% mutate(V2=sapply(V1, rxnorm::get_atc))

    #add new query to database
    rxnorm_data_ext <- rbind(rxnorm_data_ext, rxnorm_data_new_q)
    saveRDS(rxnorm_data_ext, paste0(home_directory,"rxnorm2atcdict.rda"))

    # process output for python
    rxnorm_data_out <- rxnorm_data_ext %>% filter(V1 %in% rxnorm_data_new$V1)
    rxnorm_data_out["id"] <- rownames(rxnorm_data_out)
    flatten_list <- melt(rxnorm_data_out$V2) %>% mutate(id=as.character(L1)) %>% 
                    left_join(rxnorm_data_out, by="id") %>%
                    select(V1, value) %>% mutate(V2=as.character(value)) %>% 
                    mutate(V3=V2) %>%
    #                mutate(V3=RIGHT(V2,1)) %>%
                    select(V1, V3) %>% rename(c("Rxcui"="V1", "ATC4th"="V3")) %>% unique
    write.csv(x=flatten_list, file=paste0('/home/hchan2/AKI/AKI_Python/data/',site,'/rxnorm_out_',site,'.csv'), row.names = FALSE)
    print(site)
}

In [ ]:
#sites <- list('IUR', 'MCRI', 'MCW', 'UIOWA', 'UNMC', 'UofU', 'UPITT', 'UTSW')
sites <- list('KUMC')

In [ ]:
for (site in sites){
    query_per_site_rx(site)
}

In [ ]:
query_per_site_ndc <- function(site){
    home_directory = "/home/hchan2/AKI/AKI_Python/"
    ndc_data_ext <- readRDS(paste0(home_ndirectory,"ndc2rxnorm2atcdict.rda"))
    ndc_data_new <- read.delim(paste0(home_directory,'data/',site,"/ndctmp_",site,".csv"), sep=",", stringsAsFactors=F, header = FALSE, colClasses=c('character'))

    # find new query needed
    ndc_data_q <- ndc_data_new %>% filter(!V1 %in% ndc_data_ext$V1)

    #query new data
    ndc_data_new_q <- ndc_data_q %>% mutate(V2=sapply(V1, rxnorm::from_ndc))

    ndc_data_new_q2 <- ndc_data_new_q %>% mutate(V3=sapply(V2, rxnorm::get_atc))

    #add new query to database (ndc2rxnorm2atc)
    ndc_data_ext <- rbind(ndc_data_ext, ndc_data_new_q2)
    saveRDS(ndc_data_ext, paste0(home_directory,"ndc2rxnorm2atcdict.rda"))

    ndc_data_ext <- ndc_data_ext %>% select(-V2) %>% rename(V2=V3)
    ndc_data_out <- ndc_data_ext %>% filter(V1 %in% ndc_data_new$V1)
    ndc_data_out["id"] <- ndc_data_out$V1

    flatten_list <- melt(ndc_data_out$V2) %>% mutate(id=as.character(L1)) %>% 
                    left_join(ndc_data_out, by="id") %>%
                    select(V1, value) %>% mutate(V2=as.character(value)) %>%
                    mutate(V3=V2) %>%
    #                mutate(V3=RIGHT(V2,1)) %>%
                    select(V1, V3) %>% rename(c("ndc"="V1", "ATC4th"="V3")) %>% unique

    write.csv(x=flatten_list, file=paste0('/home/hchan2/AKI/AKI_Python/data/',site,'/ndc_out_',site,'.csv'), row.names = FALSE)
    print(site)
}